# Задание 1 (3 балла)  
Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно, и с помощью модели, взятой с rusvectores (например вот этой - http://vectors.nlpl.eu/repository/20/180.zip). Обучите 2 модели по определению перефразирования на получившихся векторах и проверьте, что работает лучше. 
Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие.  
!!!! ВАЖНО: Оценивать модели нужно с помощью кросс-валидации (в семинаре не кросс-валидация)! Метрика - f1.


## Загражаем библиотеки и данные

In [2]:
!pip install pymorphy2
!pip install razdel

from lxml import html
import pandas as pd

import nltk
nltk.download('stopwords')
stops = set(nltk.corpus.stopwords.words('russian'))

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from razdel import tokenize as razdel_tokenize

from string import punctuation
punct = punctuation+'«»—…“”*№–'


def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    return ' '.join(words)

def tokenize(text):
    tokens = [token.text for token in list(razdel_tokenize(text))]
    tokens = [token for token in tokens if token.isalnum()]
    return ' '.join(tokens)


!wget https://raw.githubusercontent.com/vonCount/HSE-Computational-Linguistics-Course-2020-2021/main/paraphrases.xml
!wget https://raw.githubusercontent.com/vonCount/HSE-Computational-Linguistics-Course-2020-2021/main/corpus.xml

corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
--2021-06-28 18:19:44--  https://raw.githubusercontent.com/vonCount/HSE-Computational-Linguistics-Course-2020-2021/main/paraphrases.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3701830 (3.5M) [text/plain]
Saving to: ‘paraphrases.xml.1’

paraphrases.xml.1   100%[===================>]   3.53M  --.-KB/s    in 0.05s   

2021-06-28 18:19:44 (77.0 MB/s) - ‘paraphrases.xml.1’ saved [3701830/3701830]

--2021-06-28 18:19:44--  https://raw.githubusercontent.com/vonCount/HSE-Computational-Linguistics-Course-2020-2021/main/corpus.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.1

In [3]:
#Смотрим на данные 
data.head()

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилище решить ...,правило внесудебный проникновение полицейский ...
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


In [4]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip -O lenta.txt.zip
!unzip lenta.txt.zip
!wget http://vectors.nlpl.eu/repository/20/180.zip
!unzip 180.zip
import gensim

lenta = open('lenta.txt').read().splitlines()[:5000]
data_lenta = [normalize(text) for text in lenta]

my_w2v = gensim.models.Word2Vec([text.split() for text in data_lenta], size=50, sg=1)

ready_w2v = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

--2021-06-28 18:20:11--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5723675 (5.5M) [application/zip]
Saving to: ‘lenta.txt.zip’

lenta.txt.zip       100%[===================>]   5.46M  --.-KB/s    in 0.08s   

2021-06-28 18:20:11 (72.6 MB/s) - ‘lenta.txt.zip’ saved [5723675/5723675]

Archive:  lenta.txt.zip
replace lenta.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/._lenta.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
--2021-06-28 18:20:32--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request se

In [5]:
from collections import Counter, defaultdict
import numpy as np

def get_embedding(text, model, dim):
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total)
        except (KeyError, ValueError):
            continue
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    return vector

In [9]:
dim = 50
X_1_my_w2v = np.zeros((len(data.text_1_norm), dim))
X_2_my_w2v = np.zeros((len(data.text_2_norm), dim))

for i, text in enumerate(data.text_1_norm.values):
    X_1_my_w2v[i] = get_embedding(text, my_w2v, dim)
    
for i, text in enumerate(data.text_2_norm.values):
    X_2_my_w2v[i] = get_embedding(text, my_w2v, dim)

X_my_w2v = np.concatenate([X_1_my_w2v, X_2_my_w2v], axis=1)
y = data['label'].values

dim = 50
X_1_ready_w2v = np.zeros((len(data.text_1_norm), dim))
X_2_ready_w2v = np.zeros((len(data.text_2_norm), dim))

for i, text in enumerate(data.text_1_norm.values):
    X_1_ready_w2v[i] = get_embedding(text, ready_w2v, dim)
    
for i, text in enumerate(data.text_2_norm.values):
    X_2_ready_w2v[i] = get_embedding(text, ready_w2v, dim)

X_ready_w2v = np.concatenate([X_1_ready_w2v, X_2_ready_w2v], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
log_reg = LogisticRegression(C=1)

print('Cross val score: ', cross_val_score(log_reg, X_my_w2v, y, scoring="f1_micro"))
print('Cross val score: ', cross_val_score(log_reg, X_ready_w2v, y, scoring="f1_micro"))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Cross val score:  [0.45228216 0.4626556  0.47404844 0.3916955  0.42906574]
Cross val score:  [0.40940526 0.40940526 0.40899654 0.40899654 0.40899654]


Cross val score:  [0.45228216 0.4626556  0.47404844 0.3916955  0.42906574]
Cross val score:  [0.40940526 0.40940526 0.40899654 0.40899654 0.40899654]

В эффективности работы моделей нет больших различий

## Задание 2 (7 баллов)

Преобразуйте тексты в векторы в каждой паре 5 методами - SVD, NMF, Word2Vec (свой и русвекторовский), Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары).  

Постройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру). Попробуйте улучить метрику, изменив параметры в методах векторизации. !!УТОЧНЕНИЕ: модель нужно обучить сразу на всех 5 близостях, а не по 1 модели на каждой близости!  

SVD и NMF применяйте к данным напрямую, а w2w и fastext обучите на отдельном корпусе (как в первой части).

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
import gensim

tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data.text_1_norm, data.text_2_norm]))

svd = TruncatedSVD(100)
X_1_svd = svd.fit_transform(tfidf.transform(data.text_1_norm))
X_2_svd = svd.fit_transform(tfidf.transform(data.text_2_norm))
X_svd = np.concatenate([X_1_svd, X_2_svd], axis=1)

nmf = NMF(100)
X_1_nmf = nmf.fit_transform(tfidf.transform(data.text_1_norm))
X_2_nmf = nmf.fit_transform(tfidf.transform(data.text_2_norm))
X_nmf = np.concatenate([X_1_nmf, X_2_nmf], axis=1)

fast_text = gensim.models.FastText([text.split() for text in data_lenta], size=100, min_n=4, max_n=8) 
X_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_2_ft = np.zeros((len(data['text_2_norm']), dim))
for i, text in enumerate(data['text_1_norm'].values):
    X_1_ft[i] = get_embedding(text, fast_text, dim)
for i, text in enumerate(data['text_2_norm'].values):
    X_2_ft[i] = get_embedding(text, fast_text, dim)
X_ft = np.concatenate([X_1_ft, X_2_ft], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [14]:
from sklearn.metrics.pairwise import cosine_distances

my_w2v_dist = cosine_distances(X_1_my_w2v, X_2_my_w2v)
ready_w2v_dist = cosine_distances(X_1_ready_w2v, X_2_ready_w2v)
svd_dist = cosine_distances(X_1_svd, X_2_svd)
nmf_dist = cosine_distances(X_1_nmf, X_2_nmf)
ft_dist = cosine_distances(X_1_ft, X_2_ft)

distances = np.concatenate((my_w2v_dist, ready_w2v_dist, svd_dist, nmf_dist, ft_dist), axis=1)

log_reg2 = LogisticRegression(C=1, class_weight='balanced')

print('Cross val score: ', cross_val_score(log_reg2, distances, y, scoring="f1_micro"))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Cross val score:  [0.37136929 0.37759336 0.36816609 0.32525952 0.37647059]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Cross val score:  [0.37136929 0.37759336 0.36816609 0.32525952 0.37647059]

Результат даже ухудшился. Но я изначально планировал протестировать, что будет если у tfidf выставить 1000 features, а у всех остальных размер эмбеддинга 100

#### Изменим параметры

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
import gensim

tfidf = TfidfVectorizer(min_df=5, max_df=0.9, max_features=2000)
tfidf.fit(pd.concat([data.text_1_norm, data.text_2_norm]))

svd = TruncatedSVD(200)
X_1_svd = svd.fit_transform(tfidf.transform(data.text_1_norm))
X_2_svd = svd.fit_transform(tfidf.transform(data.text_2_norm))
X_svd = np.concatenate([X_1_svd, X_2_svd], axis=1)


nmf = NMF(200)
X_1_nmf = nmf.fit_transform(tfidf.transform(data.text_1_norm))
X_2_nmf = nmf.fit_transform(tfidf.transform(data.text_2_norm))
X_nmf = np.concatenate([X_1_nmf, X_2_nmf], axis=1)

fast_text = gensim.models.FastText([text.split() for text in data_lenta], size=50, min_n=4, max_n=8) 
dim = 200
X_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_2_ft = np.zeros((len(data['text_2_norm']), dim))
for i, text in enumerate(data['text_1_norm'].values):
    X_1_ft[i] = get_embedding(text, fast_text, dim)
for i, text in enumerate(data['text_2_norm'].values):
    X_2_ft[i] = get_embedding(text, fast_text, dim)
X_ft = np.concatenate([X_1_ft, X_2_ft], axis=1)

dim = 200
X_1_my_w2v = np.zeros((len(data.text_1_norm), dim))
X_2_my_w2v = np.zeros((len(data.text_2_norm), dim))
for i, text in enumerate(data.text_1_norm.values):
    X_1_my_w2v[i] = get_embedding(text, my_w2v, dim)
for i, text in enumerate(data.text_2_norm.values):
    X_2_my_w2v[i] = get_embedding(text, my_w2v, dim)
X_my_w2v = np.concatenate([X_1_my_w2v, X_2_my_w2v], axis=1)

dim = 200
X_1_ready_w2v = np.zeros((len(data.text_1_norm), dim))
X_2_ready_w2v = np.zeros((len(data.text_2_norm), dim))
for i, text in enumerate(data.text_1_norm.values):
    X_1_ready_w2v[i] = get_embedding(text, ready_w2v, dim)
for i, text in enumerate(data.text_2_norm.values):
    X_2_ready_w2v[i] = get_embedding(text, ready_w2v, dim)
X_ready_w2v = np.concatenate([X_1_ready_w2v, X_2_ready_w2v], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [15]:
distances = np.concatenate((my_w2v_dist, ready_w2v_dist, svd_dist, nmf_dist, ft_dist), axis=1)

In [16]:
from sklearn.linear_model import LogisticRegression
log_reg3 = LogisticRegression(C=1, class_weight='balanced')

In [17]:
from sklearn.model_selection import cross_val_score
print('Cross val score: ', cross_val_score(log_reg3, distances, y, scoring="f1_micro"))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Cross val score:  [0.37136929 0.37759336 0.36816609 0.32525952 0.37647059]


Результат опять не стал лучше. В дальнейших экспериментах надо увеличивать параметры